In [8]:
import os
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
data_dir = "Celebrity Faces Dataset/Celebrity Faces Dataset"  # clean folder
classes = sorted([f for f in os.listdir(data_dir) if not f.startswith('.')])

In [3]:
image_paths = []
labels = []

for idx, cls in enumerate(classes):
    cls_folder = os.path.join(data_dir, cls)
    for file in os.listdir(cls_folder):
        if file.lower().endswith(('.jpg', '.png', '.jpeg')):
            image_paths.append(os.path.join(cls_folder, file))
            labels.append(idx)

image_paths = np.array(image_paths)
labels = np.array(labels)

print(f"Detected {len(classes)} classes and {len(image_paths)} images.")


Detected 17 classes and 1700 images.


In [10]:
num_classes = len(np.unique(labels))
images_per_class = 100
folds = 10
images_per_fold = images_per_class // folds

In [11]:
images_by_class = []
for class_i in range(num_classes):
    images_by_class.append([])

In [12]:
for path, label in zip(image_paths, labels):
    images_by_class.append(path)

In [14]:
for class_i in range(num_classes):
    images_by_class[class_i].sort()

In [15]:
folds_by_class = []
for class_i in range(num_classes):
    class_images = images_by_class[class_i]
    class_images = images_by_class[class_i]
    class_folds = []
    for fold in range(folds):
        start = fold * images_per_fold
        end = start + images_per_fold
        fold_images = class_images[start:end]
        class_folds.append(fold_images)
    folds_by_class.append(class_folds)

In [16]:
def train(train_paths, train_labels):
    """ we need to replace this with a function that preprocessess the image and trains the model with SVM or KNN or softmax, etc"""
    model = None
    return model

In [17]:
def predict(model, test_paths):
    """this should be a script with our actual predicition logic, should return an array/list of predicted labels"""
    preds = np.zeros(len(test_paths))
    return preds